In [ ]:
import os

import numpy as np

from bclassification.utils_fcn import create_dataset
from experience import load_experience
from lib.constants import Constants as Const
from lib.data_utils import (
    make_dir,
    env_pf,
)
from lib.visualizer import Visualizer, pprint

Visualizer()

experience_dir = make_dir(os.path.join(Const.EXPERIENCE_DIR, "data-aug"))
results_dir = make_dir(os.path.join(Const.RESULTS_DIR, "_bc-fcn"))

agent_name = "agent-mip"
case_name = "l2rpn_2019_art"
env_dc = True
verbose = False

case_results_dir = make_dir(os.path.join(results_dir, f"{case_name}-{env_pf(env_dc)}"))
case, collector = load_experience(case_name, agent_name, experience_dir, env_dc=env_dc)

In [ ]:
"""
    Parameters
"""

random_seed = 0

label_mode = "dn"
n_window_targets = 12
use_actions = True

for input_mode in ["structured", "binary"]:
    for n_window_history in [1, 0]:
        for downsampling_rate in [0.05, 0.10]:
            for n_window_forecasts in [1, 0]:
                for feature_scaling in [True, False]:
                    np.random.seed(random_seed)

                    ft_str = "-ft" if feature_scaling else "" 
                    dr_str = str(int(100 * downsampling_rate))
                    f_str = str(n_window_forecasts)
                    h_str = str(n_window_history)
                    
                    file_name = f"fc-d{dr_str}-h{h_str}-f{f_str}{ft_str}-{input_mode}.npz"
                    file_path = os.path.join(experience_dir, file_name)

                    if file_name not in os.listdir(experience_dir):
                        X, Y, mask_targets, X_all, Y_all = create_dataset(
                            case,
                            collector,
                            input_mode=input_mode,
                            label_mode=label_mode,
                            n_window_history=n_window_history,
                            n_window_targets=n_window_targets,
                            downsampling_rate=downsampling_rate,
                            n_window_forecasts=n_window_forecasts,
                            use_actions=use_actions,
                            feature_scaling=feature_scaling,
                        )

                        np.savez_compressed(
                            file_path,
                            X_all=X_all,
                            Y_all=Y_all,
                            mask_targets=mask_targets,
                        )

                        pprint("Saving to:", file_path)
                    
                        del X_all
                        del Y_all
                        del X
                        del Y
                    else:
                        pprint("Done:", file_path)
        break